In [1]:
raw_data = sc.textFile('/FileStore/tables/db3yp7oo1490592954012/kddcup_data_10_percent-d8e1d.gz').cache()

<h3>Getting data Frame</h3>
<p>create dataframe out of rdd</p>

In [3]:
raw_data.take(2)

In [4]:
csv_data = raw_data.map(lambda l: l.split(","))

In [5]:
from pyspark.sql import Row
row_data = csv_data.map(lambda p: Row(
    duration=int(p[0]), 
    protocol_type=p[1],
    service=p[2],
    flag=p[3],
    src_bytes=int(p[4]),
    dst_bytes=int(p[5])
    )
)

In [6]:
interactions_df = spark.createDataFrame(row_data)
interactions_df.registerTempTable("interactions")

<b>Select tcp network interactions with more than 1 second duration and no transfer from destination</b>

In [8]:
interactions_df.show()

In [9]:
tcp_interactions = sqlContext.sql("""
    SELECT duration, dst_bytes FROM interactions WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")

In [10]:
display(tcp_interactions)

In [11]:
tcp_interactions_out = tcp_interactions.rdd.map(lambda p: "Duration: {}, Dest. bytes: {}".format(p.duration, p.dst_bytes))
for ti_out in tcp_interactions_out.collect():
  print ti_out

In [12]:
from time import time

t0 = time()
interactions_df.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()
tt = time() - t0

print "Query performed in {} seconds".format(round(tt,3))